In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing.image import load_img, img_to_array

class AIImageClassifier:
    def __init__(self, img_height=224, img_width=224, channels=3):
        self.img_height = img_height
        self.img_width = img_width
        self.channels = channels
        self.model = self.build_model()
        self.class_names = None

    def build_model(self):
        base_model = ResNet50V2(
            weights='imagenet', 
            include_top=False, 
            input_shape=(self.img_height, self.img_width, self.channels)
        )
        base_model.trainable = False
        
        model = models.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(2, activation='softmax')
        ])
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
            loss='categorical_crossentropy', 
            metrics=['accuracy', 
                     tf.keras.metrics.Precision(), 
                     tf.keras.metrics.Recall()]
        )
        return model

    def prepare_data(self, data_dir):
        datagen = ImageDataGenerator(
            rescale=1./255,
            validation_split=0.2,
            rotation_range=20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True
        )
        
        train_generator = datagen.flow_from_directory(
            data_dir,
            target_size=(self.img_height, self.img_width),  # Resize all images to specified dimensions
            batch_size=32,
            class_mode='categorical',
            subset='training',
            shuffle=True
        )
        
        validation_generator = datagen.flow_from_directory(
            data_dir,
            target_size=(self.img_height, self.img_width),  # Resize all images to specified dimensions
            batch_size=32,
            class_mode='categorical',
            subset='validation',
            shuffle=False
        )
        
        # Store class names
        self.class_names = list(train_generator.class_indices.keys())
        
        return train_generator, validation_generator

    def train(self, data_dir, epochs=50):
        train_generator, validation_generator = self.prepare_data(data_dir)
        
        early_stopping = EarlyStopping(
            monitor='val_accuracy', 
            patience=10, 
            restore_best_weights=True
        )
        
        history = self.model.fit(
            train_generator,
            epochs=epochs,
            validation_data=validation_generator,
            callbacks=[early_stopping]
        )
        return history

    def predict(self, image_path):
        # Ensure class names are set
        if self.class_names is None:
            raise ValueError("Model must be trained before prediction")
        
        # Load and preprocess image
        img = load_img(image_path, target_size=(self.img_height, self.img_width))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0
        
        # Make prediction
        prediction = self.model.predict(img_array)
        predicted_class_index = np.argmax(prediction)
        predicted_class = self.class_names[predicted_class_index]
        confidence = prediction[0][predicted_class_index] * 100
        
        return f"{predicted_class} (Confidence: {confidence:.2f}%)"

# Example Usage
if __name__ == '__main__':
    # Adjust dimensions to match your smallest dataset
    classifier = AIImageClassifier(img_height=150, img_width=150)
    classifier.train(r"C:\Users\soumy\Downloads\ai-real\AI-face-detection-Dataset")
    
    # Example prediction
    # result = classifier.predict("path/to/test/image.jpg")
    # print(result)

Found 2563 images belonging to 2 classes.
Found 640 images belonging to 2 classes.


C:\Users\soumy\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7527 - loss: 0.5394 - precision: 0.7527 - recall: 0.7527

C:\Users\soumy\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


81/81 ━━━━━━━━━━━━━━━━━━━━ 253s 3s/step - accuracy: 0.7540 - loss: 0.5367 - precision: 0.7540 - recall: 0.7540 - val_accuracy: 0.9953 - val_loss: 0.0415 - val_precision: 0.9953 - val_recall: 0.9953
Epoch 2/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 241s 3s/step - accuracy: 0.9698 - loss: 0.0893 - precision: 0.9698 - recall: 0.9698 - val_accuracy: 0.9937 - val_loss: 0.0239 - val_precision: 0.9937 - val_recall: 0.9937
Epoch 3/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 215s 3s/step - accuracy: 0.9856 - loss: 0.0518 - precision: 0.9856 - recall: 0.9856 - val_accuracy: 0.9969 - val_loss: 0.0167 - val_precision: 0.9969 - val_recall: 0.9969
Epoch 4/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 258s 3s/step - accuracy: 0.9888 - loss: 0.0339 - precision: 0.9888 - recall: 0.9888 - val_accuracy: 0.9984 - val_loss: 0.0177 - val_precision: 0.9984 - val_recall: 0.9984
Epoch 5/50
81/81 ━━━━━━━━━━━━━━━━━━━━ 303s 3s/step - accuracy: 0.9891 - loss: 0.0290 - precision: 0.9891 - recall: 0.9891 - val_accuracy: 0.9953 - val_loss: 0.0200 - val_preci

In [3]:
    # Predict single image
    result = classifier.predict(r"C:\\Users\\soumy\\Downloads\\vecteezy_ai-generated-indian-female-student_39334804.jpg")
    print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
AI Generated (Confidence: 94.52%)


In [5]:
    # Predict single image
    result = classifier.predict(r"C:\\Users\\soumy\\Downloads\\ai-generated-image.jpg")
    print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
AI Generated (Confidence: 85.57%)


In [13]:
    # Predict single image
    result = classifier.predict(r"C:\Users\soumy\Downloads\Armaan_Malik_2016.jpg")
    print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
AI Generated (Confidence: 53.08%)


In [9]:
# Predict single image
result = classifier.predict(r"C:\Users\soumy\Downloads\WhatsApp Image 2025-01-09 at 11.01.21.jpeg")
print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Real Image (Confidence: 97.87%)


In [11]:
# Example prediction
result = classifier.predict(r"C:\Users\soumy\Downloads\WhatsApp Image 2025-01-09 at 11.01.22.jpeg")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
Real Image (Confidence: 88.68%)


In [15]:
result = classifier.predict(r"C:\Users\soumy\Downloads\ai-real\AI-face-detection-Dataset\Real Image\non-child-978.png")
print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
Real Image (Confidence: 99.94%)


In [17]:
result = classifier.predict(r"C:\Users\soumy\Downloads\images.jpg")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
Real Image (Confidence: 98.94%)


In [19]:
result = classifier.predict(r"C:\Users\soumy\Downloads\no-filters-closeup-portrait-young-260nw-2103259805.webp")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
AI Generated (Confidence: 93.46%)


In [21]:
result = classifier.predict(r"C:\Users\soumy\OneDrive\Pictures\Camera Roll\WIN_20250124_22_29_33_Pro.jpg")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
Real Image (Confidence: 82.94%)


In [23]:
result = classifier.predict(r"C:\Users\soumy\Downloads\premium_photo-1661585987573-c5873c7e258e.jpg")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
AI Generated (Confidence: 64.81%)


In [27]:
result = classifier.predict(r"C:\Users\soumy\Downloads\pexels-photo-2379005.jpeg")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Real Image (Confidence: 99.45%)


In [29]:
result = classifier.predict(r"C:\Users\soumy\Downloads\v3_0924096 ai generated.jpg")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
Real Image (Confidence: 69.71%)


In [31]:
result = classifier.predict(r"C:\Users\soumy\Downloads\v3_0344789 ai generated.jpg")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
AI Generated (Confidence: 92.72%)
